# CSV to SQL

In [22]:
import pandas as pd
import numpy as np
import sys

In [23]:
def DataframeToCsv(dataframe, fileName, tableName, columns):
    numRow = len(dataframe.index)
    numCol = len(dataframe.columns)
    
    with open(fileName, 'w') as f:
        for row in range(0, numRow):
            print("INSERT INTO " + tableName + " " + columns + " VALUES (", file=f, end="")
            for column in range(0, numCol):
                print("NULL" 
                      if ((dataframe.iat[row, column] == "'nan'")
                          or (dataframe.iat[row, column] == "—")
                          or (dataframe.iat[row, column] == "'(none)'"))
                      else dataframe.iat[row, column].replace("\n", "\\n"), file=f, end="")
                if (column != numCol-1):
                    print(", ", file=f, end="")
                else:
                    print(");", file=f)

In [24]:
# PAIS TABLE
# Reads all CSVs
pais = pd.read_csv("raw-data/pais.csv")
capital = pd.read_csv("raw-data/List_of_national_capitals_1.csv")
# So I can get the continent
continents = pd.read_csv("raw-data/List_of_countries_by_GDP_(nominal)_1.csv")

# Treats CSV
continents = continents[['Country/Territory', 'UN Region']].copy()
pais.drop("Source and date", axis=1, inplace=True)
pais.drop("Area.1", axis=1, inplace=True)
pais.drop("Population density.1", axis=1, inplace=True)
pais.drop(0, axis=0, inplace=True)
pais.rename({"Country, territory or dependency" : "nome", "Population" : "populacao", "Area" : "area"}, axis=1, inplace=True)
pais = pais.sort_values("nome")
capital.drop("Notes", axis=1, inplace=True)
capital.rename({"Country/Territory" : "nome", "City/Town" : "capital"}, axis=1, inplace=True)
capital.sort_values("nome", inplace=True)
pais = pais.join(capital.set_index('nome'), on='nome')
pais = pais.join(continents.set_index('Country/Territory'), on='nome')
pais['capital'] = pais['capital'].str.replace("'", "''")
pais['populacao'] = pais['populacao'].str.replace(",", "")
pais['area'] = pais['area'].str.replace(",", "")
pais['Population density'] = pais['Population density'].str.replace(",", "")
pais.rename({"UN Region" : "continente"}, axis=1, inplace=True)
pais.update(pais[['nome', 'capital', 'continente']].applymap("'{}'".format))

In [25]:
# PIB TABLE
pib = pd.read_csv("raw-data/List_of_countries_by_GDP_(nominal)_1.csv")
# Treats CSV
pib.drop("UN Region", axis=1, inplace=True)
pib.drop("IMF.1", axis=1, inplace=True)
pib.drop("World Bank.1", axis=1, inplace=True)
pib.drop("United Nations.1", axis=1, inplace=True)
pib.drop(0, axis=0, inplace=True)
pib.drop(1, axis=0, inplace=True)
pib.rename({"Country/Territory" : "pais", "IMF" : "2022", "World Bank" : "2021", "United Nations" : "2020"}, axis=1, inplace=True)
pib['2022'] = pib['2022'].str.replace(",", "")
pib['2021'] = pib['2021'].str.replace(",", "")
pib['2020'] = pib['2020'].str.replace(",", "")
pib.update(pib[['pais']].applymap("'{}'".format))
pib = pib.sort_values("pais")
pib.head()

,pais,2022,2021,2020
120,'Afghanistan',20136,20116,19793
126,'Albania',18256,18260,14910
58,'Algeria',187155,167983,147689
205,'American Samoa',—,709,—
176,'Andorra',3302,3330,2864


In [26]:
# Governador TABLE
governador = pd.read_csv("raw-data/List_of_current_heads_of_state_and_government_1.csv")
governador2 = pd.read_csv("raw-data/List_of_current_heads_of_state_and_government_2.csv")
governador3 = pd.read_csv("raw-data/List_of_current_heads_of_state_and_government_3.csv")
# Treats CSV
governador2.drop("Associated with", axis=1, inplace=True)
governador3.drop("Also claimed by", axis=1, inplace=True)
governador = governador.append(governador2)
governador = governador.append(governador3)
#pib.drop("UN Region", axis=1, inplace=True)
#pib.drop("IMF.1", axis=1, inplace=True)
#pib.drop("World Bank.1", axis=1, inplace=True)
#pib.drop("United Nations.1", axis=1, inplace=True)
#pib.drop(0, axis=0, inplace=True)
#pib.drop(1, axis=0, inplace=True)
#pib.rename({"Country/Territory" : "pais", "IMF" : "2022", "World Bank" : "2021", "United Nations" : "2020"}, axis=1, inplace=True)
#pib['2022'] = pib['2022'].str.replace(",", "")
#pib['2021'] = pib['2021'].str.replace(",", "")
#pib['2020'] = pib['2020'].str.replace(",", "")
governador['State'] = governador['State'].str.replace("'", "''")
governador['Head of state'] = governador['Head of state'].str.replace("'", "''")
governador['Head of government'] = governador['Head of government'].str.replace("'", "''")
#governador['Head of state'] = governador['Head of state'].str.replace("President – ", "")
#governador['Head of government'] = governador['Head of government'].str.replace("President – ", "")
governador.update(governador[['State', 'Head of state', 'Head of government']].applymap("'{}'".format))
#pib = pib.sort_values("pais")
governador.head()

/tmp/ipykernel_45913/845294352.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  governador = governador.append(governador2)
/tmp/ipykernel_45913/845294352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  governador = governador.append(governador3)


,State,Head of state,Head of government
0,'Afghanistan','Supreme Leader – Hibatullah Akhundzada','Acting Prime Minister – Hasan Akhund'
1,'Albania','President – Bajram Begaj','Prime Minister – Edi Rama'
2,'Algeria','President – Abdelmadjid Tebboune','Prime Minister – Aymen Benabderrahmane'
3,'Andorra','Episcopal Co-Prince – Joan Enric Vives i Sicí...,'Prime Minister – Xavier Espot Zamora'
4,'Angola','President – João Lourenço','President – João Lourenço'


In [27]:
# Idioma TABLE
idioma = pd.read_csv("raw-data/List_of_official_languages_by_country_and_territory_1.csv")
# Treats CSV
#pib.drop("UN Region", axis=1, inplace=True)
#pib.drop("IMF.1", axis=1, inplace=True)
#pib.drop("World Bank.1", axis=1, inplace=True)
#pib.drop("United Nations.1", axis=1, inplace=True)
#pib.drop(0, axis=0, inplace=True)
#pib.drop(1, axis=0, inplace=True)
#pib.rename({"Country/Territory" : "pais", "IMF" : "2022", "World Bank" : "2021", "United Nations" : "2020"}, axis=1, inplace=True)
#pib['2022'] = pib['2022'].str.replace(",", "")
#pib['2021'] = pib['2021'].str.replace(",", "")
#pib['2020'] = pib['2020'].str.replace(",", "")
#governador['Head of state'] = governador['Head of state'].str.replace("President – ", "")
#governador['Head of government'] = governador['Head of government'].str.replace("President – ", "")
#pib = pib.sort_values("pais")
str_cols = idioma.select_dtypes(['object']).columns
idioma[str_cols] = idioma[str_cols].replace("'", "''", regex=True)
idioma.update(idioma[['Country/Region', 'Official language', 'Regional language', 'Minority language', 'National language', 'Widely spoken']].applymap("'{}'".format))
#idioma = idioma[['Country/Region', 'Official language', 'Regional language', 'Minority language', 'National language', 'Widely spoken']].replace("'", "''")
idioma.head()

,Country/Region,Official language,Regional language,Minority language,National language,Widely spoken
0,'Abkhazia','Abkhaz\nRussian','nan','Georgian','Abkhaz','nan'
1,'Afghanistan','Persian (Dari)\nPashto','Uzbek\nTurkmen\nPashayi\nNuristani\nBalochi\n...,'nan','Persian (Dari)\nPashto','Persian (Dari)'
2,'Albania','Albanian','nan','Greek\nMacedonian\nAromanian','nan','Italian'
3,'Algeria','Arabic\nTamazight','nan','nan','Arabic\nTamazight','French'
4,'Andorra','Catalan','nan','Spanish\nFrench\nPortuguese','nan','nan'


In [28]:
# Religiao TABLE
religiao = pd.read_csv("raw-data/Religions_by_country.csv")
# Treats CSV
religiao.drop("Population", axis=1, inplace=True)
religiao.drop("Christian.1", axis=1, inplace=True)
religiao.drop("Muslim.1", axis=1, inplace=True)
religiao.drop("Irreligion.1", axis=1, inplace=True)
religiao.drop("Hindu.1", axis=1, inplace=True)
religiao.drop("Buddhist.1", axis=1, inplace=True)
religiao.drop("Folk religion.1", axis=1, inplace=True)
religiao.drop("Other religion.1", axis=1, inplace=True)
religiao.drop("Jewish.1", axis=1, inplace=True)
cols = [
    "Christian",
    "Muslim",
    "Irreligion",
    "Hindu",
    "Buddhist",
    "Folk religion",
    "Other religion",
    "Jewish"
]
religiao[cols] = religiao[cols].replace(',', '', regex=True)
religiao.update(religiao[["Country"]].applymap("'{}'".format))
religiao.head()

,Country,Christian,Muslim,Irreligion,Hindu,Buddhist,Folk religion,Other religion,Jewish
0,'Angola',17237400,100000,973080,0,0,801360,0,0
1,'Cameroon',13778800,7692300,1038800,0,0,646800,529270,0
2,'Central African Republic',3938000,862000,44000,0,0,44000,0,0
3,'Chad',4559380,9200000,280750,0,0,157220,11230,0
4,"'Congo, Democratic Republic of the'",63199260,1249200,1187460,30000,0,461790,65970,2500


In [29]:
# Moeda TABLE
moeda = pd.read_csv("raw-data/List_of_circulating_currencies_1.csv")

# Treats CSV
str_cols = moeda.select_dtypes(['object']).columns
moeda[str_cols] = moeda[str_cols].replace("'", "''", regex=True)
moeda.update(moeda[[
   "State or territory",
    "Currency",
    "Symbol or\nAbbrev.",
    "ISO code", 
    "Fractional\nunit"
]].applymap("'{}'".format))
moeda.head()

,State or territory,Currency,Symbol or\nAbbrev.,ISO code,Fractional\nunit,Number\nto basic
0,'Abkhazia','Abkhazian apsar','(none)','(none)','(none)',(none)
1,'Abkhazia','Russian rouble','₽','RUB','Kopeck',100
2,'Afghanistan','Afghan afghani','Af or Afs (pl.)','AFN','Pul',100
3,'Akrotiri and Dhekelia','Euro','€','EUR','Cent',100
4,'Albania','Albanian lek','Lek','ALL','Qintar',100


In [30]:
DataframeToCsv(pais, "cooked-data/pais.sql", "Pais", "(nome, populacao, area, densidadePop, capital)")
DataframeToCsv(pib, "cooked-data/pib.sql", "PIB", "(nomePais, pib2022, pib2021, pib2020)")
DataframeToCsv(governador, "cooked-data/governador.sql", "Governador", "()")
DataframeToCsv(idioma, "cooked-data/idioma.sql", "Idioma", "()")
DataframeToCsv(religiao, "cooked-data/religiao.sql", "Religiao", "(country, christian, muslim, irreligion, hindu, buddhist, folkReligion, otherReligion, jewish)")
DataframeToCsv(moeda, "cooked-data/moeda.sql", "Moeda", "")